In [1]:
import os
import pandas as pd 
import numpy as np 
import Levenshtein
import utils

In [2]:
df1 = pd.read_csv(os.path.join('zippedData', 'tn.movie_budgets.csv.gz'))
df1['release_date'] = pd.to_datetime(df1['release_date'])
# df1['release_year'] = df1['release_date'].dt.year
df1

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,2009-12-18,Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,2019-06-07,Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,2015-05-01,Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
...,...,...,...,...,...,...
5777,78,2018-12-31,Red 11,"$7,000",$0,$0
5778,79,1999-04-02,Following,"$6,000","$48,482","$240,495"
5779,80,2005-07-13,Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5780,81,2015-09-29,A Plague So Pleasant,"$1,400",$0,$0


In [3]:
df2 = pd.read_csv(os.path.join('zippedData', 'bom.movie_gross.csv.gz'))
df2

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010
...,...,...,...,...,...
3382,The Quake,Magn.,6200.0,NaN,2018
3383,Edward II (2018 re-release),FM,4800.0,NaN,2018
3384,El Pacto,Sony,2500.0,NaN,2018
3385,The Swan,Synergetic,2400.0,NaN,2018


In [5]:
df1_titles = utils.process_strings(df1.movie).values
df2_titles = utils.process_strings(df2.title).values

In [6]:
# titles = np.tile(df2_titles, (df1_titles.size, 1)).T
titles = pd.DataFrame(index=df2_titles, columns=df1_titles)
titles.head()

,avatar,pirates of the caribbean on stranger tides,dark phoenix,avengers age of ultron,star wars ep viii the last jedi,star wars ep vii the force awakens,avengers infinity war,pirates of the caribbean at worldas end,justice league,spectre,...,newlyweds,el mariachi,primer,cavite,the mongol king,red 11,following,return to the land of wonders,a plague so pleasant,my date with drew
toy story 3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
alice in wonderland 2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
harry potter and the deathly hallows part 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
inception,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
shrek forever after,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
def similarity(data: pd.Series, base_str=None) -> pd.Series:
    strings = data.index.to_series()
    if base_str is None:
        base_str = data.name
    return strings.map(lambda x: Levenshtein.ratio(base_str, x))

In [23]:
sim_df = titles.apply(similarity)

In [48]:
match_df = sim_df.agg(['idxmax', 'max'], axis=1)
match_df.shape

(3387, 2)

In [51]:
match_df[match_df['max'] >= 0.9].sort_values('max')

,idxmax,max
bad santa 2,bad santa,0.9
planes fire rescue,planes fire and rescue,0.9
13 minutes,15 minutes,0.9
housefull 3,housefull,0.9
back to the future day,back to the future,0.9
...,...,...
silent hill revelation 3d,silent hill revelation 3d,1
lawless,lawless,1
the fiveyear engagement,the fiveyear engagement,1
the guilt trip,the guilt trip,1
